# Objective

This file collects the data used for machine learning predictions by web scrapping [sports-reference.com](sports-reference.com).

*Note:  Web Scrapping [sports-reference](sports-reference.com) should be done carefully so as not to overload their servers.*

## Set year

In [17]:
year = 2021

## Import Libraries

In [2]:
# Data Manipulation
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Web Scrapping
import requests
from bs4 import BeautifulSoup
import time

# Warnings
import warnings
warnings.filterwarnings('ignore')

/Users/mattcallicott/opt/anaconda3/envs/playground/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Data Mining Functions

`get_data`:  gets the offensive and defensive game-average statistics for each `team` and `year` combination. 
 
Returns two Pandas DataFrames in the form (offense, defense).

In [3]:
def get_data(team_name: str, year: int) -> (pd.DataFrame, pd.DataFrame):
	url = f'https://www.sports-reference.com/cfb/schools/{team_name}/{year}.html'

	page = requests.get(url)
	soup = BeautifulSoup(page.text, 'html.parser')
	
	team_table = soup.find('table', id='team')

	rows = team_table.find_all('tr')

	team_offense = [team_name] + [x.text for x in rows[2].find_all('td')]
	team_defense = [team_name] + [x.text for x in rows[3].find_all('td')]

	columns = ['team', 'g', 'pass_cmp', 'pass_att', 'pass_pct', 'pass_yds', 'pass_td', 'rush_att', 'rush_yds', 'rush_avg', 'rush_td', 'total_plays', 'total_yds', 'total_avg', 'remove_1', 'remove_2', 'remove_3', 'remove_4', 'penalty_num', 'penalty_yds', 'fumbles', 'interceptions', 'turnovers']

	team_offense_df = pd.DataFrame({i: [x] for i, x in zip(columns, team_offense)}).drop(columns=['g'])
	team_defense_df = pd.DataFrame({i: [x] for i, x in zip(columns, team_defense)}).drop(columns=['g'])

	return team_offense_df, team_defense_df

`format_team_name`:  formats a raw team name by converting the name to lowercase and replacing spaces with dashes. 
 
Returns a string. 

In [4]:
def format_team_name(raw_team_name: str) -> str:
	team_name = raw_team_name
	if '\xa0' in team_name:
		team_name = raw_team_name.split('\xa0')[1]
	team_name = team_name.replace(' ', '-')
	team_name = team_name.lower()

	return team_name

`get_schedule`:  gets the schedule for each `team` and `year` combination. 
 
Returns Pandas DataFrame that contains the home team, away team, outcome, and points for and against for each game `team` played during `year`.

In [13]:
def get_schedule(team_name: str, year: int) -> pd.DataFrame:

	page = requests.get(f'https://www.sports-reference.com/cfb/schools/{team_name}/{year}-schedule.html')
	soup = BeautifulSoup(page.text, 'html.parser')
	table = soup.find('table', id='schedule')

	data = {
		'home_team': [],
		'away_team': [],
		'outcome': [],
		'pts_for': [],
		'pts_against': []
	}


	for table_row in table.find_all('tr'):
		row_data = [x.text for x in table_row.find_all('td')]

		if row_data:
			if row_data[9]:



				home_team = format_team_name(row_data[3])
				away_team = format_team_name(row_data[5])
				loc = row_data[4]
				outcome = row_data[7]
				pts_for_obs = row_data[8]
				pts_against_obs = row_data[9]

				if loc == '@':
					home_team, away_team = away_team, home_team
					outcome = 'W' if outcome == 'L' else 'L'
				
				if outcome == 'W':
					pts_for, pts_aginst = max(pts_for_obs, pts_against_obs), min(pts_for_obs, pts_against_obs)
				else:
					pts_aginst, pts_for = max(pts_for_obs, pts_against_obs), min(pts_for_obs, pts_against_obs)

				data['home_team'].append(home_team)
				data['away_team'].append(away_team)
				data['outcome'].append(outcome)
				data['pts_for'].append(pts_for)
				data['pts_against'].append(pts_aginst)

	return pd.DataFrame(data)

## Mine Data

Get an updated list of each team sports-reference.com has by using the `year`'s standings.  

Intializes `team_names` which contains every team name.

In [6]:
page = requests.get(f'https://www.sports-reference.com/cfb/years/{year}-standings.html')
soup = BeautifulSoup(page.text, 'html.parser')
schools_table = soup.find('table', id='standings')

team_names = []

for table_row in schools_table.find_all('tr')[2:]:
	try:
		team_name = table_row.find_all('a')[0].get('href').split('/')[-2]
		team_names.append(team_name)
	except IndexError:
		continue
	except TypeError:
		continue

Iterates over `team_names` and aggregates the offensive and defensive statistics for each team in `offense_df` and `defense_df`, respectively.

In [7]:
offense_df = None
defense_df = None

for team_name in team_names:
	try:
		team_offense_df, team_defense_df = get_data(team_name, year)
		
		if offense_df is None or defense_df is None:
			offense_df = team_offense_df
			defense_df = team_defense_df
		else:
			offense_df = pd.concat((offense_df, team_offense_df))
			defense_df = pd.concat((defense_df, team_defense_df))
	except Exception as e:
		print(f"Error:  {team_name}")
	time.sleep(1)

Error:  old-dominion
Error:  connecticut


Reformats `offense_df` and `defense_df` and writes each to output.

In [8]:
offense_df = offense_df.reset_index().drop(columns=['index'])
defense_df = defense_df.reset_index().drop(columns=['index'])

offense_df.to_csv('data/offense.csv', index=False)
defense_df.to_csv('data/defense.csv', index=False)

Gets the entire college football schedule for `year` by iterating over `team_names`.  

Saves schedule as `schedule_df`.

In [18]:
schedule_df = None

for team_name in team_names:
	try:
		team_schedule_df = get_schedule(team_name, year)

		if schedule_df is None:
			schedule_df = team_schedule_df
		else:
			schedule_df = pd.concat((schedule_df, team_schedule_df))
	except Exception as e:
		print(f"Error:  {team_name}")
	time.sleep(1)

schedule_df = schedule_df.drop_duplicates().reset_index(drop=True)

In [19]:
schedule_df.to_csv(f'data/schedule_{year}.csv', index=False)

Creates the `X` feature space by iterating over `schedule_df` and joining the relevant `offense_df` and `defense_df` for both home and away teams. Only grabs relevant columns. 

Aggregates statistics by game into `game_df`, which is appended to `X` to form a complete repository.

In [ ]:
X = None


for idx, row in schedule_df.iterrows():


	home_team = row['home_team']
	away_team = row['away_team']
	win_loss_ratio = row['win_loss_ratio']

	home_offense = offense_df.query('team == @home_team')[[x for x in offense_df.columns if 'remove' not in x and x != 'team']].astype('float').reset_index(drop=True)
	home_defense = defense_df.query('team == @home_team')[[x for x in offense_df.columns if 'remove' not in x and x != 'team']].astype('float').reset_index(drop=True)
	away_offense = offense_df.query('team == @away_team')[[x for x in offense_df.columns if 'remove' not in x and x != 'team']].astype('float').reset_index(drop=True)
	away_defense = defense_df.query('team == @away_team')[[x for x in offense_df.columns if 'remove' not in x and x != 'team']].astype('float').reset_index(drop=True)

	# home_advantage = pd.concat((home_offense, away_defense)).diff(1).dropna().reset_index(drop=True)
	# away_advantage = pd.concat((away_offense, home_defense)).diff(1).dropna().reset_index(drop=True)
	
	# game_df = pd.merge(left=home_advantage, right=away_advantage, left_index=True, right_index=True, suffixes=('_home', '_away'))

	game_df = pd.merge(home_offense, home_defense, left_index=True, right_index=True, suffixes=('_home_off', '_home_def'))
	game_df = pd.merge(home_offense, home_defense, left_index=True, right_index=True, suffixes=('_home_off', '_home_def'))
	game_df = pd.merge(game_df, away_offense, left_index=True, right_index=True, suffixes=('', '_away_off'))
	game_df = pd.merge(game_df, away_defense, left_index=True, right_index=True, suffixes=('', '_away_off'))

	if game_df.shape[0] != 0:
		game_df['index'] = [idx]
		if X is None:
			X = game_df
		else:
			X = pd.concat((X, game_df))

X = X.set_index('index')

Combines `X` with relevant outcome variables. 

Separates data into final `X` and `y` DataFrames. Writes each to output.

In [ ]:
data = pd.merge(left=X, right=schedule_df[['pts_for', 'pts_against', 'outcome']], left_index=True, right_index=True)

X = data.drop(columns=['pts_for', 'pts_against', 'outcome'])
y = data['outcome'].apply(lambda x: 1 if x == 'W' else 0)

#X.to_csv(f'data/X_{year}.csv', index=False)
#y.to_csv(f'data/y_{year}.csv', index=False)